In [1]:
!gdown 1Ena8CH9H8XxY8yBx15KYmQd82lJQE6Um

Downloading...
From (original): https://drive.google.com/uc?id=1Ena8CH9H8XxY8yBx15KYmQd82lJQE6Um
From (redirected): https://drive.google.com/uc?id=1Ena8CH9H8XxY8yBx15KYmQd82lJQE6Um&confirm=t&uuid=72fb1c5a-f3f4-4471-a9f5-fe0e900c06f4
To: /kaggle/working/mlt19_yolo_obb_final.zip
100%|███████████████████████████████████████| 7.33G/7.33G [00:44<00:00, 166MB/s]


In [2]:
!unzip -q /kaggle/working/mlt19_yolo_obb_final.zip

In [3]:
%%writefile custom_yolo11s-obb.yaml

nc: 8 # IMPORTANT: Changed to 1 for the ReCTS dataset (CLASS_ID = 0)
scales: 
  # Use 'n' scale for your initial experiments to save time and resources
  n: [0.50, 0.25, 1024] # summary: 181 layers, 2624080 parameters, 2624064 gradients, 6.6 GFLOPs
  s: [0.50, 0.50, 1024] # summary: 181 layers, 9458752 parameters, 9458736 gradients, 21.7 GFLOPs
  m: [0.50, 1.00, 512] # summary: 231 layers, 20114688 parameters, 20114672 gradients, 68.5 GFLOPs
  l: [1.00, 1.00, 512] # summary: 357 layers, 25372160 parameters, 25372144 gradients, 87.6 GFLOPs
  x: [1.00, 1.50, 512] # summary: 357 layers, 56966176 parameters, 56966160 gradients, 196.0 GFLOPs

# YOLOv11 custom BACKBONE (Simplified for better Neck integration)
backbone:
  # [from, repeats, module, args]
  - [-1, 1, Conv, [64, 3, 2]]      # 0-P1/2
  - [-1, 1, Conv, [128, 3, 2]]     # 1-P2/4
  - [-1, 2, C3k2, [256, False, 0.25]] # 2
  - [-1, 1, Conv, [256, 3, 2]]     # 3-P3/8   <-- P3/8 Backbone Input
  - [-1, 2, C3k2, [512, False, 0.25]] # 4
  - [-1, 1, Conv, [512, 3, 2]]     # 5-P4/16  <-- P4/16 Backbone Input (Key/Value Source)
  - [-1, 2, C3k2, [512, True]]     # 6        <-- P4/16 Backbone Input (Alternative)
  - [-1, 1, Conv, [1024, 3, 2]]    # 7-P5/32  
  - [-1, 2, C3k2, [1024, True]]    # 8
  - [-1, 1, SPPF, [1024, 5]]       # 9
  - [-1, 1, Conv, [1024, 1, 1]]    # 10-P5/32 (Final P5 feature map) <-- P5/32 Backbone Input (Query Source)

# YOLOv11 Custom HEAD (BiFPN-Inspired Neck)
head:
  # ----------------------------------------------------------------
  # --- Stage 1: Initial Top-Down Path (FPN) ---
  
  # MOD: P5 FUSION (P5_td) - Your Cross-Attention Block
  # Fuses P5 (Query, L10) with P4 (K/V, L5). Output is 1024 channels.
  - [[5, 10], 1, CrossAttentionBlock, [1024, 512]] # 11: Fused P5 features (P5_td_in)
  - [-1, 1, MultiScaleCBAM, [1024, 16]]           # 12: Refine P5_td_in 

  # P4_td
  - [-1, 1, nn.Upsample, [None, 2, "nearest"]] # 13: Upsample P5_td_in -> P4 size
  - [[-1, 6], 1, Concat, [1]]                   # 14: cat P5_td_up and P4_in (L6)
  - [-1, 2, C3k2, [512, False]]                 # 15: P4_td (Top-down)

  # P3_td
  - [-1, 1, nn.Upsample, [None, 2, "nearest"]] # 16: Upsample P4_td -> P3 size
  - [[-1, 4], 1, Concat, [1]]                   # 17: cat P4_td_up and P3_in (L4)
  - [-1, 2, C3k2, [256, False]]                 # 18: P3_td (Top-down output)
  
  # MOD: Refine P3_td
  - [-1, 1, MultiScaleCBAM, [256, 16]]          # 19: Refine P3_td (P3/8-small)

  # ----------------------------------------------------------------
  # --- Stage 2: Bottom-Up Path (BiFPN/PANet-like) ---
  
  # P4_out (Bi-level Fusion)
  - [-1, 1, Conv, [256, 3, 2]]            # 20: Downsample P3_td (L19) -> P4 size
  # BiFPN characteristic: Concats P3_td_downsampled, P4_td (L15), and P4_in (L6)
  - [[-1, 15, 6], 1, Concat, [1]]         # 21: cat three features
  - [-1, 2, C3k2, [512, False]]           # 22: P4_out (Bottom-up output)
  
  # MOD: Refine P4_out
  - [-1, 1, MultiScaleCBAM, [512, 16]]    # 23: Refine P4_out (P4/16-medium)

  # P5_out (Bi-level Fusion)
  - [-1, 1, Conv, [512, 3, 2]]            # 24: Downsample P4_out (L23) -> P5 size
  # BiFPN characteristic: Concats P4_out_downsampled, P5_td (L12), and P5_in (L10)
  - [[-1, 12, 10], 1, Concat, [1]]        # 25: cat three features
  - [-1, 2, C3k2, [1024, True]]           # 26: P5_out (Bottom-up output)
  
  # MOD: Refine P5_out
  - [-1, 1, MultiScaleCBAM, [1024, 16]]   # 27: Refine P5_out (P5/32-large)
  
  # ----------------------------------------------------------------
  # --- OBB DETECTION LAYER ---
  # Final detection head uses the refined feature maps: P3_td (L19), P4_out (L23), P5_out (L27)
  - [[19, 23, 27], 1, OBB, [nc, 1]] # MODIFIED: Changed Detect to OBB

Writing custom_yolo11s-obb.yaml


In [4]:
%%writefile dataset.yaml
train: /kaggle/working/mlt_yolo_obb_final/images/train
val: /kaggle/working/mlt_yolo_obb_final/images/val

nc: 8

names:
  0: Arabic
  1: Latin
  2: Chinese
  3: Korean
  4: Japanese
  5: Bangla
  6: Hindi
  7: Other

Writing dataset.yaml


In [5]:
!gdown 1TgZeHp5RPPGuPPbWNUuq_qztW7owqy9T

Downloading...
From (original): https://drive.google.com/uc?id=1TgZeHp5RPPGuPPbWNUuq_qztW7owqy9T
From (redirected): https://drive.google.com/uc?id=1TgZeHp5RPPGuPPbWNUuq_qztW7owqy9T&confirm=t&uuid=e1a36666-60e7-478c-b0c7-05076cd37eeb
To: /kaggle/working/ultralytics_2.zip
100%|████████████████████████████████████████| 295M/295M [00:04<00:00, 70.2MB/s]


In [6]:
!unzip -q /kaggle/working/ultralytics_2.zip

In [2]:
%cd /kaggle/working/content/content/ultralytics

/kaggle/working/content/content/ultralytics


In [3]:
!pip install -q .

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 74.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta

In [4]:
%cd /kaggle/working

/kaggle/working


In [5]:
!ultralytics version

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
8.3.189


In [6]:
!pip install -q "numpy<2"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 77.4 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 4.4.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
google-colab 1.0.0 requires notebook==6.5.7, but you have notebook 6.5.4 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
google-colab 1.0.0 requires requests==2.32.3, but you have requests 2.32.5 which is incompatible.
google-colab 1.0.0 requires tornado==6.4.2, but you have tornado 6.5.2 which is incompatible

In [12]:
!yolo obb train data=dataset.yaml model=custom_yolo11s-obb.yaml imgsz=480 epochs=100 device=0,1 plots=True cos_lr=True batch=64 seed=42

New https://pypi.org/project/ultralytics/8.3.230 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.189 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
                                                        CUDA:1 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=dataset.yaml, degrees=0.0, deterministic=True, device=0,1, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=480, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=custom_yolo11s-obb.yaml, momentum=0.937, mosaic=

In [51]:
!zip -q -r mlt19_yolo_obb_runs.zip /kaggle/working/runs

## Evaluation

In [9]:
!gdown 1OzWkNDkIJbtfD3sbz8hpxyVDjfkr18sT

Downloading...
From (original): https://drive.google.com/uc?id=1OzWkNDkIJbtfD3sbz8hpxyVDjfkr18sT
From (redirected): https://drive.google.com/uc?id=1OzWkNDkIJbtfD3sbz8hpxyVDjfkr18sT&confirm=t&uuid=b136f59e-54d8-4268-b53d-7f0a20fc8f67
To: /kaggle/working/MLT19_val_images_t134.zip
100%|███████████████████████████████████████| 1.51G/1.51G [00:14<00:00, 101MB/s]


In [10]:
!unzip -q /kaggle/working/MLT19_val_images_t134.zip -d mlt19_val_images

In [11]:
import os
len(os.listdir("/kaggle/working/mlt19_val_images"))

2000

In [18]:
!rm /kaggle/working/mlt19_yolo_obb_final.zip
!rm /kaggle/working/MLT19_val_images_t134.zip

In [19]:
import os
import zipfile
import shutil
from ultralytics import YOLO

def save_mlt2019_obb_detections(obb_results, image_id, output_dir):
    """
    Extracts OBB coordinates and saves them in the requested format.
    Format: x1,y1,x2,y2,x3,y3,x4,y4,confidence
    """
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, f'res_{image_id}.txt')
    
    # OPEN WITH newline='\r\n' to force Windows Line Endings
    with open(output_path, 'w', newline='\r\n') as f:
        if obb_results is None:
            return # Create empty file if no detections found

        # obb_results.xyxyxyxy returns [N, 4, 2] tensor (N detections, 4 points, x/y)
        # obb_results.conf returns [N] tensor
        detections = obb_results.xyxyxyxy.cpu().numpy()
        confs = obb_results.conf.cpu().numpy()

        for poly, conf in zip(detections, confs):
            # Poly is shape (4, 2). We flatten it to [x1, y1, x2, y2, x3, y3, x4, y4]
            # We use int(round()) because MLT submission usually expects integers for coords
            points = [int(round(coord)) for point in poly for coord in point]
            
            # Create the list of 9 values (8 coords + 1 conf)
            values = points + [float(conf)]
            
            # Formatting: Coords are ints, Conf is float (4 decimal places)
            line_str = ','.join(str(v) if i < 8 else f'{v:.4f}' for i, v in enumerate(values))
            f.write(line_str + '\r\n')

def create_submission_zip(source_dir, zip_filename):
    """
    Zips files without any directory structure (Flat Zip).
    """
    print(f"Zipping files from {source_dir} to {zip_filename}...")
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(source_dir):
            for file in files:
                if file.endswith('.txt'):
                    file_path = os.path.join(root, file)
                    # arcname=file forces the file to be at the root of the zip
                    zipf.write(file_path, arcname=file)
    print("Zip creation complete.")

def predict_and_package(model_path, image_dir, txt_output_dir, zip_output_path):
    # 1. Load Model
    print(f"Loading model from: {model_path}")
    model = YOLO(model_path)

    # 2. Prepare Output Directory (Clean it first to avoid old files)
    if os.path.exists(txt_output_dir):
        shutil.rmtree(txt_output_dir)
    os.makedirs(txt_output_dir)

    # 3. Run Predictions
    print("Starting inference...")
    image_files = sorted(os.listdir(image_dir))
    
    for i, img_name in enumerate(image_files):
        if not (img_name.endswith('.jpg') or img_name.endswith('.png')):
            continue

        # PARSING ID: Assuming format "tr_img_123.jpg" -> ID is "123"
        # Adjust the index [-1] if your files are named differently
        try:
            # Splits by "_" and takes the last part ("123.jpg"), then removes extension
            image_id = img_name.split("_")[-1].split(".")[0]
        except IndexError:
            print(f"Skipping file with bad format: {img_name}")
            continue

        # Run inference
        img_path = os.path.join(image_dir, img_name)
        results = model.predict(img_path, verbose=False)
        
        # CRITICAL: Accessing OBB results, not Boxes
        obb_res = results[0].obb
        
        save_mlt2019_obb_detections(obb_res, image_id, txt_output_dir)
        
        if i % 50 == 0:
            print(f"Processed {i}/{len(image_files)} images...")

    # 4. Create Zip
    create_submission_zip(txt_output_dir, zip_output_path)
    
    # Verification Print
    print("\n--- Verification ---")
    print(f"1. Check {txt_output_dir} for text files.")
    print(f"2. Submit {zip_output_path} to the portal.")

# ==========================================
# EXECUTION BLOCK
# ==========================================

# UPDATE THESE PATHS BEFORE RUNNING
MODEL_PATH = "/kaggle/working/runs/obb/train/weights/best.pt"  # Path to your trained YOLO OBB model
VAL_IMAGES_PATH = "/kaggle/working/mlt19_val_images"
TXT_OUTPUT_DIR = "/kaggle/working/temp_txt_files"
SUBMISSION_ZIP = "/kaggle/working/submission.zip"

if __name__ == "__main__":
    predict_and_package(
        MODEL_PATH, 
        VAL_IMAGES_PATH, 
        TXT_OUTPUT_DIR, 
        SUBMISSION_ZIP
    )

Loading model from: /kaggle/working/runs/obb/train/weights/best.pt
Starting inference...
Processed 0/2000 images...
Processed 50/2000 images...
Processed 100/2000 images...
Processed 150/2000 images...
Processed 200/2000 images...
Processed 250/2000 images...
Processed 300/2000 images...
Processed 350/2000 images...
Processed 400/2000 images...
Processed 450/2000 images...
Processed 500/2000 images...
Processed 550/2000 images...
Processed 600/2000 images...
Processed 650/2000 images...
Processed 700/2000 images...
Processed 750/2000 images...
Processed 800/2000 images...
Processed 850/2000 images...
Processed 900/2000 images...
Processed 950/2000 images...
Processed 1000/2000 images...
Processed 1050/2000 images...
Processed 1100/2000 images...
Processed 1150/2000 images...
Processed 1200/2000 images...
Processed 1250/2000 images...
Processed 1300/2000 images...
Processed 1350/2000 images...
Processed 1400/2000 images...
Processed 1450/2000 images...
Processed 1500/2000 images...
Proc

In [20]:
%%writefile script.py

import os
import zipfile
import shutil
import numpy as np
from ultralytics import YOLO

def get_clockwise_points(points):
    """
    Takes a (4, 2) numpy array of points and orders them:
    Top-Left -> Top-Right -> Bottom-Right -> Bottom-Left.
    This fixes the 'Points are not clockwise' error.
    """
    # 1. Sort points based on Y-coordinate
    # (The top 2 points will be the 'top' side, bottom 2 will be 'bottom' side)
    sorted_y = points[np.argsort(points[:, 1])]
    
    top_points = sorted_y[:2]
    bottom_points = sorted_y[2:]
    
    # 2. Sort the top points by X to get TL (Left) and TR (Right)
    # TL is the one with smaller X
    if top_points[0][0] < top_points[1][0]:
        tl, tr = top_points[0], top_points[1]
    else:
        tl, tr = top_points[1], top_points[0]
        
    # 3. Sort the bottom points by X to get BL (Left) and BR (Right)
    # BL is the one with smaller X
    if bottom_points[0][0] < bottom_points[1][0]:
        bl, br = bottom_points[0], bottom_points[1]
    else:
        bl, br = bottom_points[1], bottom_points[0]
    
    # 4. Return in strict Clockwise order: TL -> TR -> BR -> BL
    return np.array([tl, tr, br, bl])

def save_mlt2019_obb_detections(obb_results, image_id, output_dir):
    """
    Saves OBB detections in 8-value format + confidence.
    Enforces Clockwise order and Windows CR/LF.
    """
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, f'res_{image_id}.txt')
    
    # newline='\r\n' forces Windows Line Endings
    with open(output_path, 'w', newline='\r\n') as f:
        if obb_results is None:
            return 

        # Get data from GPU to CPU
        detections = obb_results.xyxyxyxy.cpu().numpy()
        confs = obb_results.conf.cpu().numpy()

        for poly, conf in zip(detections, confs):
            # poly is shape (4, 2)
            
            # --- FIX: Reorder points to be strictly Clockwise ---
            try:
                ordered_poly = get_clockwise_points(poly)
                # Flatten to list: [x1, y1, x2, y2, x3, y3, x4, y4]
                points = [int(round(coord)) for point in ordered_poly for coord in point]
            except Exception as e:
                # Fallback in rare edge cases (just flatten raw)
                print(f"Warning: Sorting failed for {image_id}, using raw output. Error: {e}")
                points = [int(round(coord)) for point in poly for coord in point]
            
            # Combine points with confidence
            values = points + [float(conf)]
            
            # Format: 8 integers, 1 float (4 decimal places)
            # Example: 100,200,300,200,300,400,100,400,0.9500
            line = ','.join(str(v) if i < 8 else f'{v:.4f}' for i, v in enumerate(values))
            f.write(line + '\r\n')

def create_submission_zip(source_dir, zip_filename):
    """
    Zips the text files into a FLAT structure (no folders inside).
    """
    print(f"Creating submission zip: {zip_filename}")
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(source_dir):
            for file in files:
                if file.endswith('.txt'):
                    file_path = os.path.join(root, file)
                    # arcname=file ensures it sits at the root of the zip
                    zipf.write(file_path, arcname=file)
    print("Zip created successfully.")

def process_images(model_path, image_dir, txt_output_dir, zip_output_path):
    # 1. Load Model
    print(f"Loading YOLO model: {model_path}")
    model = YOLO(model_path)

    # 2. Prepare Output Directory (Clean start)
    if os.path.exists(txt_output_dir):
        shutil.rmtree(txt_output_dir)
    os.makedirs(txt_output_dir)

    # 3. Predict Loop
    image_files = sorted(os.listdir(image_dir))
    print(f"Found {len(image_files)} images. Starting inference...")
    
    for i, img_name in enumerate(image_files):
        # Filter for images only
        if not img_name.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            continue

        # Extract ID safely
        # Handles filenames like "tr_img_00969.jpg" -> "00969"
        try:
            image_id = img_name.split("_")[-1].split(".")[0]
        except Exception:
            print(f"Skipping weird filename: {img_name}")
            continue

        # Inference
        img_path = os.path.join(image_dir, img_name)
        results = model.predict(img_path, verbose=False)
        
        # Get OBB component
        obb_res = results[0].obb
        
        # Save
        save_mlt2019_obb_detections(obb_res, image_id, txt_output_dir)

        # Progress log
        if i > 0 and i % 50 == 0:
            print(f"Processed {i} images...")

    # 4. Zip results
    create_submission_zip(txt_output_dir, zip_output_path)
    print("\n--- DONE ---")
    print(f"Submit this file: {zip_output_path}")

# =========================================================
# CONFIGURATION - UPDATE THESE PATHS
# =========================================================
if __name__ == "__main__":
    # Path to your trained weights
    MODEL_WEIGHTS = "/kaggle/working/runs/obb/train/weights/best.pt"  
    
    # Path to the folder containing validation images
    VAL_IMAGES_DIR = "/kaggle/working/mlt19_val_images" 
    
    # Temporary folder to store the .txt files before zipping
    TEMP_TXT_DIR = "/kaggle/working/temp_txt_files"
    
    # Final Output Zip file name
    FINAL_ZIP_PATH = "/kaggle/working/submission.zip"

    # Run the main process
    process_images(MODEL_WEIGHTS, VAL_IMAGES_DIR, TEMP_TXT_DIR, FINAL_ZIP_PATH)

Writing script.py


In [21]:
!python script.py

Loading YOLO model: /kaggle/working/runs/obb/train/weights/best.pt
Found 2000 images. Starting inference...
Processed 50 images...
Processed 100 images...
Processed 150 images...
Processed 200 images...
Processed 250 images...
Processed 300 images...
Processed 350 images...
Processed 400 images...
Processed 450 images...
Processed 500 images...
Processed 550 images...
Processed 600 images...
Processed 650 images...
Processed 700 images...
Processed 750 images...
Processed 800 images...
Processed 850 images...
Processed 900 images...
Processed 950 images...
Processed 1000 images...
Processed 1050 images...
Processed 1100 images...
Processed 1150 images...
Processed 1200 images...
Processed 1250 images...
Processed 1300 images...
Processed 1350 images...
Processed 1400 images...
Processed 1450 images...
Processed 1500 images...
Processed 1550 images...
Processed 1600 images...
Processed 1650 images...
Processed 1700 images...
Processed 1750 images...
Processed 1800 images...
Processed 18

In [7]:
from ultralytics import YOLO
model = YOLO("/kaggle/working/runs/obb/train/weights/best.pt")

In [21]:
import os
import easyocr
import numpy as np
import cv2 
from PIL import Image
import re

# 1. Initialize EasyOCR Readers
class_map = {
    0: "Arabic", 1: "Latin", 2: "Chinese", 3: "Korean",
    4: "Japanese", 5: "Bangla", 6: "Hindi", 7: "Other"
}

lang_map = {
    0 : ["ar","en"], 1 : ["la"], 2 : ["ch_sim","en"], 3 : ["ko","en"],
    4 : ["ja","en"], 5 : ["bn","en"], 6 : ["hi","en"], 7 : ["la"]
}

READERS = {}
print("Initializing Readers...")
for cls_id, langs in lang_map.items():
    try:
        # Try GPU, fall back to CPU
        READERS[cls_id] = easyocr.Reader(langs, gpu=True)
    except:
        READERS[cls_id] = easyocr.Reader(langs, gpu=False)
print("Readers Initialized.")

# 2. Helper Functions

def ensure_clockwise(box_points):
    """
    Calculates the 'edge sum' used by the evaluation script.
    If the sum is positive, the points are Counter-Clockwise (or wrong order).
    We flip them to ensure they are Clockwise (Negative sum).
    """
    edge_sum = 0
    for i in range(4):
        p1 = box_points[i]
        p2 = box_points[(i + 1) % 4]
        edge_sum += (p2[0] - p1[0]) * (p2[1] + p1[1])
    
    # If sum > 0, it's counter-clockwise according to the eval script logic.
    if edge_sum > 0:
        return box_points[::-1]
    return box_points

def get_rotated_crop(image_np, box_points):
    """Crops and straightens the oriented bounding box."""
    box_points = np.array(box_points, dtype="float32")
    
    # Width (Top & Bottom edges)
    width_a = np.sqrt(((box_points[0][0] - box_points[1][0]) ** 2) + ((box_points[0][1] - box_points[1][1]) ** 2))
    width_b = np.sqrt(((box_points[2][0] - box_points[3][0]) ** 2) + ((box_points[2][1] - box_points[3][1]) ** 2))
    max_width = max(int(width_a), int(width_b))

    # Height (Left & Right edges)
    height_a = np.sqrt(((box_points[0][0] - box_points[3][0]) ** 2) + ((box_points[0][1] - box_points[3][1]) ** 2))
    height_b = np.sqrt(((box_points[1][0] - box_points[2][0]) ** 2) + ((box_points[1][1] - box_points[2][1]) ** 2))
    max_height = max(int(height_a), int(height_b))

    dst_pts = np.array([
        [0, 0],
        [max_width - 1, 0],
        [max_width - 1, max_height - 1],
        [0, max_height - 1]
    ], dtype="float32")

    M = cv2.getPerspectiveTransform(box_points, dst_pts)
    warped = cv2.warpPerspective(image_np, M, (max_width, max_height))
    return warped

def text_extractor(img, reader):
    text_results = reader.readtext(img, detail=0)
    if not text_results:
        return "###"
    elif len(text_results) == 1:
        return text_results[0].strip() or "###"
    else:
        joined = " ".join(t.strip() for t in text_results if t.strip())
        return joined or "###"

def save_mlt2019_detections(image_dir, obb_results, image_id, output_dir, filename, readers):
    os.makedirs(output_dir, exist_ok=True)
    # Ensure filename matches "res_ID.txt" format
    output_path = os.path.join(output_dir, f'res_{image_id}.txt')
    
    image_pil = Image.open(os.path.join(image_dir, filename))
    image_np = np.array(image_pil) 

    # Create file (even if empty) to ensure 1:1 mapping
    # "newline='\r\n'" ensures Windows line endings as requested
    with open(output_path, 'w', encoding='utf-8', newline='\r\n') as f:
        if obb_results is None:
            return

        coords = obb_results.xyxyxyxy.cpu().numpy()
        confs = obb_results.conf.cpu().numpy()
        classes = obb_results.cls.cpu().numpy()

        for box_points, conf, cls_id in zip(coords, confs, classes):
            # 1. Enforce Clockwise Order
            box_points = ensure_clockwise(box_points)
            
            # 2. Extract Text
            cls_id = int(cls_id)
            reader = readers[cls_id]
            cropped_img = get_rotated_crop(image_np, box_points)
            text = text_extractor(cropped_img, reader)
            
            # 3. Format Line: x1,y1,x2,y2,x3,y3,x4,y4,confidence,transcription
            quad_flat = box_points.flatten().astype(int).tolist()
            
            line_parts = []
            # Add coords (8 values)
            line_parts.extend([str(c) for c in quad_flat])
            # Add confidence (1 value)
            line_parts.append(f'{conf:.4f}')
            # Add transcription (1 value)
            line_parts.append(str(text))
            
            # Join with commas
            line = ",".join(line_parts)
            f.write(line + '\r\n')

def predict_val_images(model, image_dir, output_dir, readers):
    valid_exts = ('.png', '.jpg', '.jpeg')
    files = sorted([f for f in os.listdir(image_dir) if f.lower().endswith(valid_exts)])
    
    print(f"Processing {len(files)} images...")
    
    for img_name in files:
        # Robust ID extraction: grab the number part specifically
        # Assumes filenames like "img_123.jpg" or "tr_img_0123.jpg"
        # We look for the LAST sequence of digits in the name usually
        match = re.search(r'([0-9]+)(?=[^0-9]*$)', os.path.splitext(img_name)[0])
        if match:
            image_id = match.group(1)
        else:
            # Fallback if no digits found
            image_id = os.path.splitext(img_name)[0]

        # Predict
        results = model.predict(os.path.join(image_dir, img_name), verbose=False)
        obb_results = results[0].obb 
        
        save_mlt2019_detections(image_dir, obb_results, image_id, output_dir, img_name, readers)
    
    print("Prediction complete.")

# 3. Execution (Update paths as needed)
predict_val_images(model, "/kaggle/working/mlt19_val_images", "/kaggle/working/mlt19_e2e_yolo11_val_image_predictions", READERS)

Initializing Readers...
Readers Initialized.
Processing 2000 images...
Prediction complete.


In [23]:
import os
len(os.listdir("/kaggle/working/mlt19_e2e_yolo11_val_image_predictions"))

2000

In [24]:
sorted(os.listdir("/kaggle/working/mlt19_e2e_yolo11_val_image_predictions"))[0]

'res_00001.txt'

In [25]:
import zipfile
import os

# 1. Define your directories
# Make sure this matches where you saved your .txt files in the previous step
source_dir = "/kaggle/working/mlt19_e2e_yolo11_val_image_predictions" 
output_zip_path = "/kaggle/working/mlt19_e2e_submission.zip"

def create_flat_zip(source, output):
    # Create the zip file
    with zipfile.ZipFile(output, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Loop through files in the source directory
        for filename in os.listdir(source):
            # Filter to ensure we only grab the text files
            if filename.endswith(".txt"):
                file_path = os.path.join(source, filename)
                
                # CRITICAL STEP: 
                # 'arcname' is the name the file will have INSIDE the zip.
                # By setting it to just 'filename', we strip all folder paths.
                zipf.write(file_path, arcname=filename)
                
    print(f"Successfully created: {output}")
    print(f"Number of files in zip: {len(zipfile.ZipFile(output).namelist())}")

# Run the function
create_flat_zip(source_dir, output_zip_path)

Successfully created: /kaggle/working/mlt19_e2e_submission.zip
Number of files in zip: 2000


## Test Evaluation

In [50]:
!gdown 1lJuKQ2iY--OKPzS4btCdMDLObAzTmAMu
!gdown 140aSNoVw8bdYIo3BZhDd-KIpuXNDDa5V

Downloading...
From (original): https://drive.google.com/uc?id=1lJuKQ2iY--OKPzS4btCdMDLObAzTmAMu
From (redirected): https://drive.google.com/uc?id=1lJuKQ2iY--OKPzS4btCdMDLObAzTmAMu&confirm=t&uuid=a95b47ca-50bf-4a71-9bbf-dc3ddefeae71
To: /kaggle/working/MLT19_TestImagesPart1.zip
100%|███████████████████████████████████████| 3.47G/3.47G [00:17<00:00, 201MB/s]
Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.
	Check FAQ in https://github.com/wkentaro/gdown?tab=readme-ov-file#faq.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=140aSNoVw8bdYIo3BZhDd-KIpuXNDDa5V

but Gdown can't. Please check connections and permissions.


In [42]:
!unzip -q /kaggle/working/MLT19_TestImagesPart1.zip
!rm /kaggle/working/MLT19_TestImagesPart1.zip
!unzip -q /kaggle/working/MLT19_TestImagesPart2.zip
!rm /kaggle/working/MLT19_TestImagesPart2.zip

In [45]:
!mkdir mlt19_test_images

In [46]:
len(os.listdir("/kaggle/working/TestImagesPart1")),len(os.listdir("/kaggle/working/TestImagesPart2"))

(5000, 5000)

In [47]:
import shutil

for file in sorted(os.listdir("/kaggle/working/TestImagesPart1")):
    shutil.copy(os.path.join("/kaggle/working/TestImagesPart1",file),"/kaggle/working/mlt19_test_images")
shutil.rmtree("/kaggle/working/TestImagesPart1")

for file in sorted(os.listdir("/kaggle/working/TestImagesPart2")):
    shutil.copy(os.path.join("/kaggle/working/TestImagesPart2",file),"/kaggle/working/mlt19_test_images")
shutil.rmtree("/kaggle/working/TestImagesPart2")

In [48]:
len(os.listdir("/kaggle/working/mlt19_test_images"))

5000

Successfully created: /kaggle/working/mlt19_e2e_submission.zip
Number of files in zip: 2000
